# Imports

In [45]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

# basics
import numpy as np
import pandas as pd
import requests as r
import re
import geopandas as gpd
import json

# viz
import matplotlib.pyplot as plt
import seaborn as sns

# utils
import os
import time
from tqdm import tqdm
from geopy.geocoders import Nominatim
import geopandas as gpd
import pgeocode
import urllib
from shapely.geometry import Point
from itertools import chain
from kedro.config import ConfigLoader
from ppm.utils.external_get_data import (
    find_regex_cep,
    search_cep,
    find_census_area_by_zip,
    get_connection,
    scrapping_zipimoveis,
    find_cbg
)
from ppm.utils.readers import (
    reader,
    unzip_file
)
from ppm.utils.feature_process import (
    replace_names,
    transform_float,
    convert_to_float,
    replace_na_with_mean,
    replace_na_with_mode
)

# Parameters

In [2]:
uf = 'PB'
path_root_oos = os.path.join(
    '..', 'data', # data or oos
)
path_root_data = os.path.join(
    '..', 'data', # data or oos
)
path_raw = os.path.join(
    path_root_data, '01_raw'
)
path_intermediate = os.path.join(
    path_root_oos, '02_intermediate'
)
path_intermediate_data = os.path.join(
    path_root_data, '02_intermediate'
)
path_primary = os.path.join(
    path_root_oos, '03_primary'
)
root_brasilapi = 'https://brasilapi.com.br/api'
root_zap = 'https://www.zapimoveis.com.br/'
url_path_zapimoveis = os.path.join(
    root_zap, 'venda/imoveis/pb+joao-pessoa/?pagina={}'
)
path_census_data = os.path.join(
    path_raw, 'PB_20171016'
)
path_census_data_csv = os.path.join(
    path_census_data,
    f'{uf}',
    f'Base informaçoes setores2010 universo {uf}',
    'CSV'
)
path_columns_cat = os.path.join(
    path_intermediate, "categorical_dict"
)

file_path_data_merged = os.path.join(
    path_intermediate, 'scrapping_data_concat.csv'
)
file_path_data_shp = os.path.join(
    path_raw, 
    'PB_Setores_2021',
    'PB_Setores_2021.shp'
)
file_path_shp_processed = os.path.join(
    path_intermediate_data, "data_shp_processed"
)
file_path_census_processed = os.path.join(
    path_intermediate_data, "data_census_processed.csv"
)
file_path_processed = os.path.join(
    path_primary, "data_processed.csv"
)
file_path_processed_concat = os.path.join(
    path_primary, "data_processed_concat.csv"
)
file_path_data_input = os.path.join(
    path_primary, "data_input.csv"
)
file_path_categorical_replaces = os.path.join(
    path_columns_cat, "{}.json"
)

# Read datasets

In [3]:
# unzip
if not os.path.exists(path_census_data):
    unzip_file(
        path_census_data+'.zip',
        path_census_data
    )

In [4]:
file_path_census_data = {
    i.split(".")[0]: os.path.join(path_census_data_csv, i)for i in os.listdir(path_census_data_csv) 
}

In [5]:
data_census_merged = pd.read_csv(file_path_census_processed)
data_shp = gpd.read_file(file_path_shp_processed)

/tmp/ipykernel_637364/1582892236.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,

# Scrapping

In [6]:
n_pages_init = 1
n_pages_final = 61 # 101

In [7]:
data_results_scrapping = {}
for i in tqdm(range(n_pages_init, n_pages_final + 1)):
    print(url_path_zapimoveis.format(i))
    data_results_scrapping[
        f'page_n{i}'
    ] = scrapping_zipimoveis(url_path_zapimoveis.format(i))
    time.sleep(10)

  0%|                                                                                                                                                                                       | 0/61 [00:00<?, ?it/s]

https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa/?pagina=1
- number of locals found: [99]



  2%|██▊                                                                                                                                                                          | 1/61 [01:08<1:08:02, 68.05s/it]

https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa/?pagina=2
- number of locals found: [99]



  3%|█████▋                                                                                                                                                                       | 2/61 [02:15<1:06:48, 67.94s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 59/61 [58:04<01:50, 55.07s/it]

https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa/?pagina=60
- number of locals found: [0]

trying: [1]
trying: [2]
trying: [3]
trying: [4]
trying: [5]
trying: [6]


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 60/61 [58:30<00:46, 46.37s/it]

https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa/?pagina=61
- number of locals found: [100]



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [59:35<00:00, 58.62s/it]


In [8]:
content_final = {}
errors = []
for page, content in tqdm(data_results_scrapping.items()):
    data_tmp = {}
    for idx, i in enumerate(list(content.items())):
        data_tmp[f'n{idx}'] = {}
        data_tmp[f'n{idx}']['ID'] = i[0].replace('id-','')
        data_tmp[f'n{idx}']['price'] = transform_float(i[1]['price'][0])
        data_tmp[f'n{idx}']['local'] = i[1]['local']
        data_tmp[f'n{idx}']['highlight'] = i[1]['highlight']
        for f_name, f_value in i[1]['features'].items():
            data_tmp[f'n{idx}'][f_name] = replace_names(f_value)
        data_tmp[f'n{idx}'] = pd.DataFrame(data_tmp[f'n{idx}'])
    try:
        content_final[page] = pd.concat(list(data_tmp.values()),
                                        ignore_index = True)
    except Exception as e:
        errors.append((page, e))
content_final = pd.concat(list(content_final.values()),
                          ignore_index = True).replace('', np.nan)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:04<00:00, 13.83it/s]


In [9]:
cols_selected = [
    'areas',
    'bedrooms',
    'parking-spaces',
    'bathrooms'
]
string_rows = content_final[cols_selected].apply(lambda x: x.str.contains('-', na = False))

for i in cols_selected:
    values_changed = content_final[
        string_rows[i]
    ][i].apply(
        lambda x: (int(x.split("-")[0])+int(x.split("-")[1]))/2
    )
    content_final.loc[values_changed.index, i] = values_changed.values

In [10]:
ceps = []
for i in tqdm(content_final['local']):
    try:
        ceps.append( search_cep(i)[1] )
    except:
        ceps.append( np.nan )
content_final['cep'] = ceps
content_final['cep'] = content_final['cep'].fillna(np.nan)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4792/4792 [1:15:17<00:00,  1.06it/s]


# Pre-processing

In [11]:
cols_to_drop = [
    "bathrooms"
]
which_rows_to_drop = [
    'areas', 
    'bedrooms',
    'parking-spaces',
    'highlight'
]
id_tag = [
    "ID"
]
content_final_process = content_final.copy()

In [12]:
content_final_process.price = content_final_process.price.apply(convert_to_float)

In [13]:
lats = []
longs = []
cbg = []
nomi = pgeocode.Nominatim('br')
for local in  tqdm(content_final_process.local):
    if local!=None:
        address = local + f', {uf}, Brasil'
        try:
                                                                                                                        
            geolocator = Nominatim(user_agent="geolocalização")
            location = geolocator.geocode(address)

            lats.append(float(location.latitude))
            longs.append(float(location.longitude))

        except:
            lats.append(np.nan)
            longs.append(np.nan)

    else:
        lats.append(np.nan)
        longs.append(np.nan)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4792/4792 [1:16:08<00:00,  1.05it/s]


In [14]:
content_final_process['lats'] = lats
content_final_process['longs'] = longs

In [15]:
# Criar os objetos de ponto com as latitudes e longitudes
pontos = [Point(xy) for xy in zip(content_final_process['longs'], content_final_process['lats'])]

In [16]:
cbgs = find_cbg(data_shp, pontos)
content_final_process['cd_setor'] = cbgs

  0%|                                                                                                                                                                                     | 0/4792 [00:00<?, ?it/s]/home/manuel/.cache/pypoetry/virtualenvs/property-price-modelling-UIhV6JXy-py3.10/lib/python3.10/site-packages/shapely/predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4792/4792 [00:02<00:00, 2348.79it/s]


In [17]:
# order columns
id_columns = [
    "ID",
    "cd_setor"
]
content_final_process = content_final_process[id_columns+[i for i in content_final_process.columns if i not in id_columns]]
content_processed = content_final_process.dropna(subset = ["cd_setor"]).reset_index(drop = True)

In [18]:
content_processed.cd_setor = content_processed.cd_setor.astype(str)
data_census_merged.cd_setor = data_census_merged.cd_setor.astype(str)

## Save scrapping

In [19]:
content_processed.to_csv(file_path_processed)

# Concat census

In [20]:
data_merged = content_processed.merge(
    data_census_merged, 
    on = ["cd_setor"],
    how = "left"
)
data_merged = data_merged.merge(
    data_shp[["cd_setor","nm_sit"]],
    on = ["cd_setor"],
    how="left"
)
data_merged.insert(
    4,
    "bairro",
    data_merged.local.apply(
        lambda x: x.split(", ")[-1]
    )
)

In [21]:
data_merged.columns = data_merged.columns.str.replace(" ","_")
columns_na = list(data_merged.isna().sum()[data_merged.isna().sum()!=0].index)

In [22]:
data_bairros, errors = replace_na_with_mean(data_merged, columns_na, 'bairro')
data_merged_not_nan = pd.concat(list(data_bairros.values()), ignore_index = True)
data_merged_not_nan = data_merged_not_nan[
    ~data_merged_not_nan['ID'].duplicated()
].reset_index(drop = True)

 34%|█████████████████████████████████████████████████████████▏                                                                                                               | 1383/4088 [00:03<00:06, 450.05it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4088/4088 [00:07<00:00, 527.14it/s]


 54%|███████████████████████████████████████████████████████████████████████████████████████████▏                                                                             | 2205/4088 [00:03<00:02, 707.94it/s]


 13%|██████████████████████▌                                                                                                                                                   | 542/4088 [00:01<00:11, 318.22it/s]


 54%|██████████████████████████████████████████████████████████████████████████████████████████▌                                                                              | 2191/4088 [00:05<00:03, 545.49it/s]


...

 56%|█████████████████████████████████████████████████████████████████████████████████████████████▉                                                                          | 2287/4088 [00:01<00:01, 1207.02it/s]


 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                    | 3196/4088 [00:02<00:00, 1354.08it/s]


  0%|                                                                                                                                                                                     | 0/4088 [00:00<?, ?it/s]


 24%|█████████████████████████████████████████▏                                                                                                                               | 995/4088 [00:00<00:02, 1380.67it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [05:15<00:00,  5.18s/it]


In [23]:
data_merged_not_nan.drop(
    data_merged_not_nan.columns[data_merged_not_nan.columns.str.contains("situacao")],
    axis = 1,
    inplace = True)

## Save data processed

In [24]:
data_merged_not_nan.to_csv(file_path_processed_concat)

# Drop columns with problemns

In [25]:
cods_cols = list(data_merged_not_nan.columns[data_merged_not_nan.columns.str.contains("cod")])
other_cols = [
    "cep",
    "lats",
    "longs",
    "local",
    "nm_sit"
]
all_columns = cods_cols + other_cols
data_merged_not_nan.drop(all_columns,
                         axis = 1, 
                         inplace = True)

# Categorical columns process

In [26]:
if not os.path.exists(path_columns_cat):
    os.makedirs(path_columns_cat)

In [27]:
cat_cols_nome = [
    'nome_grande_regiao_basico_pb',
    'nome_da_uf__basico_pb',
    'nome_da_meso_basico_pb',
    'nome_da_micro_basico_pb',
    'nome_da_rm_basico_pb',
    'nome_do_municipio_basico_pb',
    'nome_do_distrito_basico_pb',
    'nome_do_subdistrito_basico_pb',
]
data_merged_not_nan.drop(cat_cols_nome, axis = 1, inplace = True)

In [28]:
cat_cols = [
    "bairro"
] + [
    "nome_do_bairro_basico_pb",
    "highlight"
]

cat_dict_all = {}
for col in cat_cols:
    cat_dict = {}
    for idx, i in enumerate(data_merged_not_nan[col].unique()):
        cat_dict[i] = idx
    cat_dict_all[col] = cat_dict

In [29]:
data_merged_not_nan, e3 = replace_na_with_mode(
    data_merged_not_nan,
    ['highlight'],
    'bairro')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 97.54it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 108.29it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 113.87it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.95it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 395.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 142.78it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 217.86it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 96.52it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [30]:
for column_name, content_cat in cat_dict_all.items():
    data_merged_not_nan[column_name].replace(cat_dict_all[column_name], inplace = True)

In [31]:
data_final = data_merged_not_nan.copy()
data_final = data_final.replace("X", np.nan)

### Save categorical labels

In [47]:
for i_col_cat, i_content_cat in cat_dict_all.items():
    with open(file_path_categorical_replaces.format(i_col_cat), 'w') as file:
        json.dump(i_content_cat, file)

### Process column with problem

In [32]:
for pro_col in data_final.columns:
    try:
        data_final[pro_col] = data_final[pro_col].astype(float)
    except:
        data_final[pro_col] = data_final[pro_col].astype(str).str.replace(",",".").astype(float)

## Drop bathrooms

In [33]:
data_final.drop("bathrooms", axis = 1, inplace = True)

## Drop last nans

In [34]:
data_final = data_final.dropna().reset_index(drop = True)

# Save dataset final

In [35]:
data_final.to_csv(file_path_data_input)